In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('mushrooms.csv')

In [7]:
df

class cap-shape cap-surface cap-color bruises odor gill-attachment  \
0        p         x           s         n       t    p               f   
1        e         x           s         y       t    a               f   
2        e         b           s         w       t    l               f   
3        p         x           y         w       t    p               f   
4        e         x           s         g       f    n               f   
...    ...       ...         ...       ...     ...  ...             ...   
8119     e         k           s         n       f    n               a   
8120     e         x           s         n       f    n               a   
8121     e         f           s         n       f    n               a   
8122     p         k           y         n       f    y               f   
8123     e         x           s         n       f    n               a   

     gill-spacing gill-size gill-color  ... stalk-surface-below-ring  \
0               c         n          k  ...                        s   
1               c         b          k  ...                        s   
2               c         b          n  ...                        s   
3               c         n          n  ...                        s   
4               w         b          k  ...                        s   
...           ...       ...        ...  ...                      ...   
8119            c         b          y  ...                        s   
8120            c         b          y  ...                        s   
8121            c         b          n  ...                        s   
8122            c         n          b  ...                        k   
8123            c         b          y  ...                        s   

     stalk-color-above-ring stalk-color-below-ring veil-type veil-color  \
0                         w                      w         p          w   
1                         w                      w         p          w   
2                         w                      w         p          w   
3                         w                      w         p          w   
4                         w                      w         p          w   
...                     ...                    ...       ...        ...   
8119                      o                      o         p          o   
8120                      o                      o         p          n   
8121                      o                      o         p          o   
8122                      w                      w         p          w   
8123                      o                      o         p          o   

     ring-number ring-type spore-print-color population habitat  
0              o         p                 k          s       u  
1              o         p                 n          n       g  
2              o         p                 n          n       m  
3              o         p                 k          s       u  
4              o         e                 n          a       g  
...          ...       ...               ...        ...     ...  
8119           o         p                 b          c       l  
8120           o         p                 b          v       l  
8121           o         p                 b          c       l  
8122           o         e                 w          v       l  
8123           o         p                 o          c       l  

[8124 rows x 23 columns]

In [8]:
X = pd.get_dummies(df.drop(columns='class'))
y = df['class']

In [309]:
def normalize(X):
    X = np.asarray(X)
    X_normalized = X / np.linalg.norm(X, axis=1, keepdims=True)
    return X_normalized

In [310]:
X = normalize(X)

In [9]:
def train_test_split(X, y, test_size=0.25, shuffle=True):
    X, y = np.asarray(X), np.asarray(y)
    n = len(X)
    if shuffle:
        perm = np.random.permutation(n)
        X, y = X[perm], y[perm]
    test_rows = round(test_size * n)
    X_train, X_test, y_train, y_test = X[test_rows:], X[:test_rows], y[test_rows:], y[:test_rows]
    return X_train, X_test, y_train, y_test

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [19]:
class KNNClassifier:
    def __init__(self, k=5):
        self._k = k
    
    def __euclidean_distance(self, x):
        dist = np.sqrt(((x - self._X) ** 2).sum(1))
        return dist
    
    def fit(self, X, y):
        self._X = X
        self._y = y
    
    def predict(self, X):
        dists = np.array(list(map(lambda row: self.__euclidean_distance(row), X)))
        knn_idxs = np.argpartition(dists, self._k)[:, :self._k]
        knn = self._y[knn_idxs]
        pred = np.array(list(map(lambda row: max(row, key=lambda s: (row == s).sum()), knn)))
        return pred
    
    def score(self, X, y):
        preds = self.predict(X)
        correct = (preds == y).sum()
        accuracy = correct / y.size
        return accuracy

In [20]:
clf = KNNClassifier()

In [21]:
clf.fit(X_train, y_train)

In [22]:
clf.score(X_test, y_test)